In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import simple_pe
from simple_pe.localization import event
from pesummary.utils.samples_dict import SamplesDict
from astropy.cosmology import Planck18 as cosmo
import numpy as np
from simple_pe.detectors import network
from simple_pe import io
import pandas as pd
import h5py

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


## S250818t

In [12]:
params_vals = {
        'ra': 0.32,
        'dec': -1.3,
        'iota': 1.04,
        'psi': 2.7,
        'phase': 0.7,
        'geocent_time': 1439518156.375,
        'chirp_mass_source': 11.16,
        'mass_ratio': 0.49,
        'comoving_distance': 993.61,
        'redshift': 0.24
    }

params = {key: np.array([val]) for key, val in params_vals.items()}

samples = SamplesDict(params)
samples.generate_all_posterior_samples()
samples.generate_all_posterior_samples()

2025-08-25  19:31:24 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find f_low in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find reference_frequency in input file. Using 20Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2025-08-25  19:31:24 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.0039062

In [13]:
ncs = io.generate_noise_curves_from_names(['AdvVirgo', 'aLIGOZeroDetHighPower'], [20]*2)
for nc in ncs.values():
    nc.generate_reach_bandwidth_vs_mchirp(grid_points=30, 
                                          mc_min = 0.95 * samples['chirp_mass'].min(),
                                          mc_max = 1.05 * samples['chirp_mass'].max())

2025-08-25  19:31:28 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default            | 0/30 [00:00<?, ?it/s]
2025-08-25  19:31:28 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  19:31:29 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default    | 1/30 [00:00<00:21,  1.36it/s]
2025-08-25  19:31:29 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2025-08-25  19:31:29 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default    | 2/30 [00:01<00:21,  1.31it/s]
2025-08-25  19:31:29 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default
2

In [14]:
ncs.update(io.generate_noise_curves_from_files([], 
                                               ['next_gen_detect/noise_curves/Asharp-asd.txt'], [10]))

In [15]:
ncs

{('aLIGOZeroDetHighPower',
  20): <simple_pe.detectors.noise_curves.NoiseCurve at 0x7164c4189190>,
 ('AdvVirgo',
  20): <simple_pe.detectors.noise_curves.NoiseCurve at 0x71644f0f1b10>,
 ('next_gen_detect/noise_curves/Asharp-asd.txt',
  10): <simple_pe.detectors.noise_curves.NoiseCurve at 0x71644f11ac50>}

In [48]:
netw1 = network.Network()
for ifo, psd in zip(['L1', 'H1', 'V1'], ['aLIGOZeroDetHighPower']*2 + ['AdvVirgo']):
    netw1.add_ifo_from_noise_curve(ifo, ncs[(psd,20)])
netw2 = network.Network()
for ifo, psd in zip(['L1', 'H1', 'I1'], ['aLIGOZeroDetHighPower']*3):
    netw2.add_ifo_from_noise_curve(ifo, ncs[(psd,20)])

In [49]:
for event_num in range(len(samples['chirp_mass'])):
    my_event1 = event.Event.from_samples_dict(samples[event_num:event_num+1])
    my_event1.add_network(netw1)

In [50]:
for event_num in range(len(samples['chirp_mass'])):
    my_event2 = event.Event.from_samples_dict(samples[event_num:event_num+1])
    my_event2.add_network(netw2)

In [51]:
my_event1.localize_all()
my_event2.localize_all()

In [52]:
my_event1.area

{'time': 40.88601568092482,
 'coh': 29.129627572052954,
 'left': 9.776696132346135,
 'right': 11.737070639498748,
 'marg': 9.83700236080931}

In [53]:
my_event2.area

{'time': 2253.9880031864486,
 'coh': 2253.988003186448,
 'left': 1605.4406208050566,
 'right': 1658.8982791652884,
 'marg': 1615.8949803094993}